In [18]:
import pandas as pd
import os
import numpy as np
import logging
import sys
from datetime import datetime, timedelta

### Load environment vars and directories

In [11]:
KEY_DIR = os.path.join("/Users/felisialoukou/Documents/","govuk-network-data", "key")
KEY_PATH = os.path.join(KEY_DIR, os.listdir(KEY_DIR)[0])
PROJECT_ID = "govuk-bigquery-analytics"

#### Logging for `pandas_gbq`

In [12]:
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

# Extract page-hit only user journeys for 2-3 weeks back
8.8 GB
### Define date range
    > 3 wks before, up to current date

In [37]:
N_weeks = 3
yesterday = 1
N_days = (N_weeks*7) + yesterday

start = (datetime.now() - timedelta(days=N_days)).date().strftime("%Y%m%d")
end = (datetime.now() - timedelta(days=yesterday)).date().strftime("%Y%m%d")

print(datetime.now())
print(start, end)

2019-04-18 12:07:19.967568
20190327 20190417


In [58]:
daterange = "_{}_{}_".format(start, end)
daterange

'_20190327_20190327_'

In [ ]:
## Query

In [59]:
query = """SELECT
  COUNT(*) AS Occurrences,
  PageSeq_Length,
  PageSequence,
  CIDSequence
FROM (
  SELECT
    *
  FROM (
    SELECT
      CONCAT(fullVisitorId,"-",CAST(visitId AS STRING),"-",CAST(visitNumber AS STRING)) AS sessionId,
      STRING_AGG(IF(htype = 'PAGE',
          pagePath,
          NULL),">>") OVER (PARTITION BY fullVisitorId, visitId, visitStartTime ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS PageSequence,
       STRING_AGG(IF(htype = 'PAGE',
          content_id,
          NULL),">>") OVER (PARTITION BY fullVisitorId, visitId, visitStartTime ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS CIDSequence,
      SUM(IF(htype='PAGE',
          1,
          0)) OVER (PARTITION BY fullVisitorId, visitId, visitStartTime ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS PageSeq_Length
    FROM (
      SELECT
        fullVisitorId,
        visitId,
        visitNumber,
        visitStartTime,
        hits.page.pagePath AS pagePath,
        hits.hitNumber AS hitNumber,
        hits.type AS htype,
        (
        SELECT
          value
        FROM
          hits.customDimensions
        WHERE
          index=4) AS content_id,
        (
        SELECT
          value
        FROM
          hits.customDimensions
        WHERE
          index=2) AS document_type
      FROM
        `govuk-bigquery-analytics.87773428.ga_sessions_*` AS sessions
      CROSS JOIN
        UNNEST(sessions.hits) AS hits
      WHERE
        _TABLE_SUFFIX BETWEEN '_start_'
        AND '_end_' )
    WHERE
      pagePath != '/'
      AND document_type NOT IN (
        'document_collection',
        'finder',
        'homepage',
        'license_finder',
        'mainstream_browse_page',
        'organisation',
        'search',
        'service_manual_homepage',
        'service_manual_topic',
        'services_and_information',
        'taxon',
        'topic',
        'topical_event',
        'fatality_notice',
        'contact',
        'service_sign_in',
        'html_publication',
        'calculator',
        'completed_transaction' )
      AND content_id NOT IN ( '00000000-0000-0000-0000-000000000000',
        '[object Object]'))
  WHERE
    PageSeq_Length >1
  GROUP BY
    sessionId,
    PageSequence,
    CIDSequence,
    PageSeq_Length)
GROUP BY
  PageSequence,
  CIDSequence,
  PageSeq_Length"""

In [60]:
query = query.replace("_start_", start).replace("_end_", end)
query

'SELECT\n  COUNT(*) AS Occurrences,\n  PageSeq_Length,\n  PageSequence,\n  CIDSequence\nFROM (\n  SELECT\n    *\n  FROM (\n    SELECT\n      CONCAT(fullVisitorId,"-",CAST(visitId AS STRING),"-",CAST(visitNumber AS STRING)) AS sessionId,\n      STRING_AGG(IF(htype = \'PAGE\',\n          pagePath,\n          NULL),">>") OVER (PARTITION BY fullVisitorId, visitId, visitStartTime ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS PageSequence,\n       STRING_AGG(IF(htype = \'PAGE\',\n          content_id,\n          NULL),">>") OVER (PARTITION BY fullVisitorId, visitId, visitStartTime ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS CIDSequence,\n      SUM(IF(htype=\'PAGE\',\n          1,\n          0)) OVER (PARTITION BY fullVisitorId, visitId, visitStartTime ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS PageSeq_Length\n    FROM (\n      SELECT\n        fullVisitorId,\n        visitId,\n       

# Extract data from BigQuery

In [56]:
df_in = pd.read_gbq(query,
                       project_id=PROJECT_ID,
                       reauth=False,
                       private_key=KEY_PATH,
                       dialect="standard")

df_in.shape

Requesting query... 
Query running...
Job ID: f875a606-d968-4667-99a2-8115efae7808
  Elapsed 6.44 s. Waiting...
  Elapsed 7.87 s. Waiting...
  Elapsed 9.31 s. Waiting...
  Elapsed 10.74 s. Waiting...
  Elapsed 12.01 s. Waiting...
  Elapsed 13.29 s. Waiting...
  Elapsed 14.73 s. Waiting...
  Elapsed 16.17 s. Waiting...
  Elapsed 17.6 s. Waiting...
  Elapsed 19.03 s. Waiting...
  Elapsed 20.47 s. Waiting...
  Elapsed 21.81 s. Waiting...
  Elapsed 23.23 s. Waiting...
  Elapsed 24.67 s. Waiting...
Query done.
Processed: 1.5 GB Billed: 1.5 GB
Standard price: $0.01 USD

Got 1069433 rows.

Total time taken 138.99 s.
Finished at 2019-04-18 12:25:30.


(1069433, 3)

### Add `Page_List` column

In [ ]:
pagelist = [pageseq.split(">>") for pageseq in df_in['PageSequence'].values]
df_in['Page_List'] = pagelist

### Save out 

In [ ]:
bq_dir = os.path.join(os.getenv("DATA_DIR"),"raw", "bq_journey_extract")
# bq_file = os.path.join(bq_dir, "pageseq_user_journey_"+date_range+".csv.gz")
# page_views_file = os.path.join(bq_dir, "pageviews_"+date_range+".csv.gz")
# df_in.to_csv(bq_file, compression="gzip", sep='\t', index=False)

In [ ]:
### Drop infrequent journeys
bq_file_doo = os.path.join(bq_dir, "pageseq_user_journey"+date_range+"doo.csv.gz")
df_in[df_in.Occurrences>1].to_csv(bq_file_doo, compression="gzip", sep='\t', index=False)

# Make network data